<h1><b>Solving the Travelling Salesman Problem (TSP) by the Genetic Algorithm (GA)</b></h1>
<hr>
<h2><b>The Travelling Salesman Problem (TSP)</b></h2>
<p1>It is defined as:<br>

<i><strong>Given a set of waypoints and distances between them, find the shortest possible route that visits each waypoint once, then return to the initial waypoint</strong></i>

</p1>

The TSP is a minimization problem, and the objective function is stated as the following:

\begin{align*}
Minimize:
\sum_{i=1}^{n} \sum_{j=1}^{n} d_{ij}\;x_{ij}
\end{align*}

<br>

\begin{align*}
Subject\;to:
    \begin {aligned}
        \sum_{j=1}^{n} x_{ij} = 1,\;j = 1, 2, ..., n\\
        \sum_{i=1}^{n} x_{ij} = 1,\;i = 1, 2, ..., n
    \end {aligned}
\end{align*}

<br>

where $d_{ij}$ denotes the distance between city $i$ and $j$ <br>
$\;\;\;\;\;\;\;\;$ $x_{ij}$ denotes a deicision variable with the following definition:

\begin{align*}
x_{ij} = 
\left\{
    \begin {aligned}
         & 1, \quad & if\;directed\;from\;city\;i\;to\;city\;j\\
         & 0, \quad & otherwise                 
    \end{aligned}
\right.
\end{align*}

And the search space increases exponentially:<br>
<center><img src="search_space.png"/></center>

<br>

As we can obeserve from the figure above, there are numerous possible soultions with the increasing number of cities. 

Searching for an exact solution for larger instances of TSP using traditional approaches(brute force, integer programming, dynamic programming) is infeasible. 

Metaheuristic algorithms are developed and designed in terms of efficiency to tackle complex optimization problems. 

It provides near-optimal solution to optimization problems, while maintaining short computational time. 

Therefore, in this project, we are using the __Genetic Algorithm (GA)__ - a metaheuristic algorithm to solve the TSP.

<hr>

<h2>Import Libraries</h2>

Install the following libraries to your python pip before importing them

 [folium](https://pypi.org/project/folium/) | [googlemaps](https://pypi.org/project/googlemaps/) | [pandas](https://pypi.org/project/pandas/) | [polyline](https://pypi.org/project/polyline/)

In [41]:
import csv
import datetime
import folium
import googlemaps 
import math
import pandas as pd
import polyline
import random
import time

<hr>

<h2>Dataset</h2>

In this project, the TSP we are solving is [the optimal driving route through 50 U.S. Landmarks](https://randalolson.com/2015/03/08/computing-the-optimal-road-trip-across-the-u-s/) suggested by Olson (2015)

And the following code reads the csv file containing station address and coordinates

In [32]:
geo_data = pd.read_csv("50_Landmarks.csv")
stations = geo_data["Station"]  # extract the "station" column from the csv

Before we proceed to use the GA to solve this problem, we need a distance matrix for GA to compute the route for us.

<hr>

<h2>Using the Google Maps API Service to Get Distance Matrix</h2>

To retrieve the distance matrix, we are using the [Google Maps API](https://developers.google.com/maps) to get the driving route distances between each stations.

As we have 50 stations in the TSP, there should be 2450 route distances (excluding the diagonal of zeros) we have to request from Google Maps.

__Before using any API from Google Maps, you have to register for an [API key](https://support.google.com/googleapi/answer/6158862?hl=en)__

The following cell is the setup of using Google Maps API.

In [33]:
api_key = "" # Paste your own api key here
gmaps = googlemaps.Client(key=api_key)
t = datetime.datetime(year=2024, month=12, day=31, hour=0, minute=0, second=0)  # Remember to set a future date and time to ensure the API service works

As there is a limited amount of data being requested per request as a free user, we have to split it into 2 requests.

__(Run the following cell if you want to generate your own distance matrix. Else, just use the provided distance matrix)__

In [ ]:
dm = [] 

for k in range(len(stations)):
    request1 = gmaps.distance_matrix(origins=stations[k], # Requesting the distances of the first 25 combinations between each stations
                                     destinations=stations[:], 
                                     units="metric", 
                                     mode="driving", 
                                     departure_time=t)
    request2 = gmaps.distance_matrix(origins=stations[k], # Requesting the distances of the remaining combinations
                                     destinations=stations[25:len(stations)], 
                                     units="metric", 
                                     mode="driving", 
                                     departure_time=t)

    row = []
    for element in request1["rows"][0]["elements"]:
        row.append(element["distance"]["value"]) 

    for element in request2["rows"][0]["elements"]:
        row.append(element["distance"]["value"])

    dm.append(row)

with open("50_Landmarks_Distance_Matrix.csv", "w", newline="") as file:  # write the distance matrix into a csv file
    writer = csv.writer(file)
    writer.writerows(dm)

<hr>

<h2>Distance Matrix</h2>

For the ease of programme design and computation, we handle the distance martix as a 2D list.

Moreover, to get the same result, it is suggested to use the same csv file provided.

In [47]:
dm_data = "50_Landmarks_Distance_Matrix.csv"

with open(dm_data, newline="") as csvfile: # Read and convert distance matrix into a 2D list
    reader = csv.reader(csvfile, quoting=csv.QUOTE_NONNUMERIC)
    dis_mtx = list(reader)

<hr>

<h2>Genetic Algorithm (GA)</h2>

GA mimics the evolutionary process of the nature and generalize it as an optimization framework. 

It has an advantage of being highly flexible to solve various optimization problems.

GA is mainly composed by:
<ls>
    <li>Initilization - Initialize a population of candidates (solutions) to perform optimization</li>
    <li>Selection - A operator to select candidates (solutions) with better fitness(objective function value)</li>
    <li>Crossover - A operator to mate selected candidates (solutions) and reproduce offsprings {new candidates (solution) containing mixed information from parents}</li>
    <li>Mutation - A operator to perform random jump by amending candidates randomly</li>
</ls>

Although the GA is very fast in solving TSP, it does not guarantee for finding the optimal solution.

The following cell is an GA based algorithm without the crossover operator:

In [43]:
class GA:

    def __init__(self, distance_matrix:list[int], population_size:int, num_generations:int, mutation_rate:int) -> None:  # Initialize data structures and parameters
        self.dm = distance_matrix
        self.pop = population_size  # number of candidates in the population
        self.generations = num_generations  # number of iterations that GA runs
        self.mutate = mutation_rate  # how many candidates will be mutated
        self.stations = list(range(len(distance_matrix)))  # a list containing 0 - 49 for solution generation
        self.population = []  # empty list to contain candidates
        self.pool = []  # empty list to contain candidates and mutated candidates
        self.best = ()  # empty set to contain the best candidate of each iteration


    def generate_routes(self) -> None:
        """Solution generator"""
        stations = self.stations[:]  # copy the station list
        for i in range(self.pop):
            random.shuffle(stations)  # randomly generate a solution (route)
            self.population.append((stations[:], self.total_distance(stations[:])))  # store the solution and its fitness


    def total_distance(self, route:list[int]) -> float:
        """Calculator of the total distance of a single route"""
        length = 0
        for i in range(len(self.dm)):  # starts calculating from the last index in the route
            node_1 = route[i-1]
            node_2 = route[i]
            length += self.dm[node_1][node_2]
        return length

        
    def selection(self) -> None:
        """Select the top 10% of candidates in the current population"""
        sorted_population = sorted(self.population, key=lambda x: x[1], reverse=False)  # sort the candidates base on their fitness, in ascending order
        self.pool = sorted_population[:int(math.ceil(self.pop / 10))]  # select the top 10% and put in the pool
        self.best = self.pool[0]  # find the best candidate
        self.population = self.pool[:]  # replace population with pool to temporarily save them for reference in mutation 

    
    def mutation(self) -> None:
        """Mutate candidates in the pool"""
        for candidate in self.population:
            
            for k in range(self.mutate):  # Shuffle the route randomly, for self.mutate times
                route = candidate[0][:]
                start_index = random.randint(0, len(route) - 1)  # randomly pick a starting point
                length = random.randint(2, len(self.dm) // 2)  # randomly pick a segment length

                subset = route[start_index:start_index + length]  # slice the segment
                route = route[:start_index] + route[start_index + length:]  # remove the segment from route

                insert_index = random.randint(0, len(route) + len(subset) - 1)  # randomly pick an index to insert the sliced segment
                route = route[:insert_index] + subset + route[insert_index:]  # insert the segment
                self.pool.append((route, self.total_distance(route)))  # store the mutated route in the pool
                
        self.population = self.pool[:]  # store a copy in population
        self.pool = []  # clear pool to release memory


    def run(self) -> list|float|float:
        """Runs the GA"""
        start_time = time.time()  # record starting time
        self.generate_routes()
        for k in range(self.generations):  # iteration process
            self.selection()
            self.mutation()
        
        converted_route = [stations[index] for index in self.best[0]]  # convert the route from list[int] into list[str] to show station address
        end_time = time.time()  # record end time
        time_diff = end_time - start_time  # compute the computational time
        return converted_route, self.best[1], time_diff

<hr>

<h2>Execution of GA</h2>

The population size of GA is set to be 100, number of generations is set to be 3000, and the mutation rate is 9.

So, there will be 100 routes performing selection and mutation in each iteration. 

And each selected route will mutate 9 times. 

The whole iteration process will be run 3000 times.

In [52]:
ga = GA(distance_matrix=dis_mtx, population_size=100, num_generations=3000, mutation_rate=9)
route, total_distance, computational_time = ga.run()
print(f"The best route found: {route}")
print(f"The total distance: {total_distance}m")
print(f"Time consumed: {computational_time}s")

The best route found: ['Maryland State House, 100 State Cir, Annapolis, MD 21401', 'The Mark Twain House & Museum, Farmington Avenue, Hartford, CT', 'Congress Hall, Congress Place, Cape May, NJ 08204', 'Statue of Liberty, Liberty Island, NYC, NY', 'Pikes Peak, Colorado', 'The Breakers, Ochre Point Avenue, Newport, RI', 'USS Constitution, Boston, MA', 'Acadia National Park, Maine', 'Omni Mount Washington Resort, Mount Washington Hotel Road, Bretton Woods, NH', 'Shelburne Farms, Harbor Road, Shelburne, VT', 'Liberty Bell, 6th Street, Philadelphia, PA', 'New Castle Historic District, Delaware', 'Lost World Caverns, Lewisburg, WV', 'Spring Grove Cemetery, Spring Grove Avenue, Cincinnati, OH', 'Olympia Entertainment, Woodward Avenue, Detroit, MI', 'Mammoth Cave National Park, Mammoth Cave Pkwy, Mammoth Cave, KY', 'West Baden Springs Hotel, West Baden Avenue, West Baden Springs, IN', 'Lincoln Home National Historic Site Visitor Center, 426 South 7th Street, Springfield, IL', 'Gateway Arch, W

<hr>

<h2>Visualization of Route on Map</h2>

In this part, we will be visualizing the route found by GA previously. 

The base map we are using is the OpenStreetMap, which is open source and free.

First, we will be marking the stations on the map.

In [44]:
map = folium.Map(location=[38, -96], tiles="OpenStreetMap", zoom_start=5) # Initialize a empty map centred at US

for i in range(len(stations)):
    lat, lng = geo_data["Latitude"][i], geo_data["Longitude"][i]
    marker = folium.Marker(location=[lat, lng], popup=stations[i]).add_to(map) # Add markers (stations) to map

Run the following cell if you want to show the marked map

In [ ]:
map

Then, we will get the driving route polyline from Google Maps Directions API.

However, it is encoded and we have to decode it in order to plot it by using folium.

In [53]:
"""Get the encoded polyline by using the Google Maps Directions API, to visualize the best route computed from the SIB algorithm"""
polyline_list = []  # list for storing polyline data

for i in range(len(route)):
    result = gmaps.directions(origin=route[i-1], 
                              destination=route[i], 
                              units="metric", 
                              departure_time=t)
    steps = result[0]["legs"][0]["steps"]  # only obtain the polyline data from Directions API
    for j in range(len(steps)):
        polyline_list.append(steps[j]["polyline"]["points"])  # store the polyline

for polyline_data in polyline_list:
    decoded_polyline = polyline.decode(polyline_data)  # decode the polyline
    folium.PolyLine(locations=decoded_polyline, color='blue').add_to(map)  # plot the polylines on the marked map

map  # shows the plotted map

<hr>

<h2>Save the Plotted Map</h2>

In [ ]:
map.save("Best_route.html")  # save the map as html 

<hr>